# Train an MLP-based propaganda detector that uses BERT text representations as input
In this notebook we will develop an MLP based propaganda detector, which uses BERT to vectoriz the input title-sentence pair. Note that the BERT model itself will *not* be updated during training.

In [1]:
# load data from CW2 to train and test the model
import pandas as pd
from sklearn.utils import shuffle
df = pd.read_table('../CW2/coursework2_train.tsv')
df = shuffle(df) # randomly shuffle data entries 
df

,article_id,article_title,label,sentence_text
7520,711622457,UK: Labour MP Cites Ban of Robert Spencer & Pa...,non-propaganda,— Donald J. Trump (@realDonaldTrump) November ...
6719,765913191,US Conference of Mayors Call For More Gun Conf...,propaganda,"The facts refute their arguments, but beyond t..."
9916,705409419,﻿Vatican Theologian Sacked for Questioning “Me...,non-propaganda,Can we not see in Fr.
517,771879020,DOJ Surrenders: 3D Print Gun Files Are Protect...,propaganda,Cody finally won against the federal beast aft...
9310,758469195,American in China injured in 'sonic attack' si...,non-propaganda,Some have permanent hearing loss or concussion...
...,...,...,...,...
4297,787529309,The Last-Minute Character Assassination of Jud...,non-propaganda,"""Senator Grassley must postpone the vote until..."
9849,731063195,One Trillion Stars,non-propaganda,"Photo credit: Keilana, Roberta Mura"
1981,790266787,Avenatti’s Freak Show,non-propaganda,But I have never done that to her or to anyone.
1994,790266787,Avenatti’s Freak Show,non-propaganda,An abused person wouldn’t do that.


In [2]:
raw_labels = df.label.values.tolist()
docs = df.sentence_text.values.tolist()
titles = df.article_title.values.tolist()

label_dic = {'non-propaganda':0, 'propaganda':1}

assert len(docs) == len(raw_labels) == len(titles)
labels = [label_dic[rl] for rl in raw_labels] # transfer raw labels (strings) to integer numbers
print('total data size: {}, label type num: {}'.format(len(docs), len(label_dic)))

np_num = len([ll for ll in labels if ll == 0])
p_num = len([ll for ll in labels if ll == 1])
print(np_num, p_num)

total data size: 11464, label type num: 2
8227 3237


In [3]:
# split the data into train, dev and test
import random

train_ratio = 0.6
dev_ratio = 0.2
train_idx = []
dev_idx = []
test_idx = []
for i in range(len(docs)):
    rnd = random.random()
    if rnd < train_ratio: train_idx.append(i)
    elif rnd < train_ratio+dev_ratio: dev_idx.append(i)
    else: test_idx.append(i)

print('train size {}, dev size {}, test size {}'.format(len(train_idx), len(dev_idx), len(test_idx)))

train size 6923, dev size 2217, test size 2324


In [4]:
# now we use the BERT model to vectorize all sentence-title pairs
# !pip install transformers 
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
from tqdm import tqdm

gpu = True
bert_type = 'large'
if bert_type == 'base':
    bert_dim = 768
else: 
    bert_dim = 1024
use_titles = True # whether to consider the titles when making predictions
bert_batch_size = 10

all_input_vecs = []
bert_tokenizer = BertTokenizer.from_pretrained('bert-{}-uncased'.format(bert_type))
bert_model = BertModel.from_pretrained('bert-{}-uncased'.format(bert_type))

if gpu:
    bert_model.to('cuda')

for i in tqdm(range(0, len(docs))):
    if use_titles:
        sent = [[titles[i], docs[i]]]
    else:
        sent = [docs[i]]
    input_to_bert = bert_tokenizer.batch_encode_plus(sent)['input_ids']
    input_to_bert = torch.tensor(input_to_bert)
    if gpu: input_to_bert = input_to_bert.to('cuda')
    words_vecs = bert_model(input_to_bert)[0][:,1:,:].squeeze().cpu().detach().numpy()
    sent_vec = np.mean(words_vecs, axis=0)
    # print(sent_vec.shape)
    all_input_vecs.append(sent_vec)

100%|██████████| 11464/11464 [04:37<00:00, 41.32it/s]


In [5]:
all_input_vecs = np.array(all_input_vecs)
print(all_input_vecs.shape)

(11464, 1024)


In [6]:
# then we define the RNN-based classifier
import torch
import torch.nn as nn

class MLP_Clf(nn.Module):
    def __init__(self, input_dim, cls_num, gpu):
        super(MLP_Clf, self).__init__()
        self.bert_dim = input_dim
        # MLP structure: three layers in total, dim of the hidden layer is the same as input layer
        self.fc1 = nn.Linear(self.bert_dim, self.bert_dim) 
        self.atv_fnc = nn.ReLU()
        self.fc2 = nn.Linear(self.bert_dim, cls_num)
        # use gpu or not
        self.gpu = gpu
        if self.gpu:
            self.to('cuda')
    def forward(self, input_batch, input_titles=None):
        if self.gpu:
            input_batch = input_batch.to('cuda')
        logits = self.fc2( self.atv_fnc( self.fc1(input_batch) ) )
        return logits


In [7]:
# because the training data is inbalanced, we use simple down-sampling to balance the data
# used to train the model
import random
import numpy as np
def down_sample():
    np_idx = [i for i in train_idx if labels[i]==0]
    p_idx = [i for i in train_idx if labels[i]==1]
    each_cat_num = min(len(np_idx), len(p_idx))
    random.shuffle(np_idx)
    random.shuffle(p_idx)
    wanted_idx = np_idx[:each_cat_num] + p_idx[:each_cat_num]
    random.shuffle(wanted_idx)
    return wanted_idx
    
wanted_idx = down_sample()

Before we train our MLP model, we first check the performance of some simple baseline methods.

In [8]:
# random baseline
from sklearn.metrics import precision_recall_fscore_support
rand_pred = [random.randint(0,1) for i in range(len(test_idx))]
pre, rec, f1, _ = precision_recall_fscore_support(np.array(labels)[test_idx], rand_pred,average='macro')
print('\n---> the macro-F1 of random baseline on dev set is {}'.format(f1))


---> the macro-F1 of random baseline on dev set is 0.47265816526516913


In [9]:
# majority baseline
major_pred = [0]*len(test_idx)
pre, rec, f1, _ = precision_recall_fscore_support(np.array(labels)[test_idx], major_pred, average='macro')
print('\n---> the macro-F1 of majority baseline on dev set is {}'.format(f1))


---> the macro-F1 of majority baseline on dev set is 0.41608040201005025


/home/cim/staff/uhac002/PycharmProjects/ScratchPad/venv_nlp/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Now we start to train our MLP model with BERT vectors as input. 

In [10]:
model = MLP_Clf(bert_dim, len(label_dic), gpu)
loss_fnc = torch.nn.CrossEntropyLoss() # cross entropy loss

# hyper parameters
n_epochs = 100 # number of epoch (i.e. number of iterations)
batch_size = 32
lr = 1e-4 # initial learning rate

# init optimizer and scheduler (lr adjustor)
import torch.optim as optim
optimizer = optim.Adam(params=model.parameters(), lr=lr) # use Adam as the optimizer
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8) # after each epoch, the learning rate is discounted to its 95%

In [13]:
# before we train the model, we first look at its initial performance on the test set
# without performing any training

import numpy as np

with torch.no_grad(): # let pytorch know that no gradient should be computed
    model.eval() # let the model know that it in test mode, i.e. no gradient and no dropout
    predictions = []
    for i in range(0,len(dev_idx),batch_size):
        idx = dev_idx[i:i+batch_size] 
        vecs = torch.tensor(all_input_vecs[idx])
        if gpu: vecs = vecs.to('cuda')
        y_pred = model(vecs).cpu().detach().numpy()
        pred_labels = [np.argmax(entry) for entry in y_pred]
        predictions += pred_labels
    pre, rec, f1, _ = precision_recall_fscore_support(np.array(labels)[dev_idx], predictions,average='macro')
    print('\n---> before training, the macro-F1 on dev set is {}'.format(f1))
    print('pred 1 percent', np.sum(predictions)/len(predictions))


---> before training, the macro-F1 on dev set is 0.727675023081504
pred 1 percent 0.2372575552548489


In [14]:
best_f1 = -1.
best_model = None
import copy
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from nltk.tokenize import word_tokenize
from tqdm import tqdm

for epoch_i in tqdm(range(n_epochs)):
    # the inner loop is over the batches in the dataset
    model.train() # let pytorch know that gradients should be computed, so as to update the model
    ep_loss = []
    sample_idx = down_sample()
    for i in range(0,len(train_idx),batch_size):
        # Step 0: Get the data
        idx = train_idx[i:i+batch_size] 
        if len(idx) == 0: break
        # print(idx)
        vecs = torch.tensor(all_input_vecs[idx])
        if gpu: vecs = vecs.to('cuda')
        target_labels = list(np.array(labels)[idx])
        # print(sents[0])
        y_target = torch.tensor(target_labels, dtype=torch.int64).squeeze()
        if gpu:
            y_target = y_target.to('cuda')
        
        # Step 1: Clear the gradients 
        optimizer.zero_grad()

        # Step 2: Compute the forward pass of the model
        y_pred = model(vecs)
        # print(y_pred)
        yp = y_pred.cpu().detach().numpy()
        pred_labels = [np.argmax(entry) for entry in yp]
        # print('pred labels', pred_labels)
        # print('true labels', y_target)

        # Step 3: Compute the loss value that we wish to optimize
        loss = loss_fnc(y_pred, y_target)
        # print(loss)
        ep_loss.append(loss.cpu().detach().numpy())

        # Step 4: Propagate the loss signal backward
        loss.backward()

        # Step 5: Trigger the optimizer to perform one update
        optimizer.step()
    
    print('\n======epoch {} loss======'.format(epoch_i),np.mean(ep_loss))
    
    # after each epoch, we can test the model's performance on the test set
    with torch.no_grad(): # let pytorch know that no gradient should be computed
        model.eval() # let the model know that it in test mode, i.e. no gradient and no dropout
        predictions = []
        for i in range(0,len(dev_idx),batch_size):
            idx = dev_idx[i:i+batch_size] 
            vecs = torch.tensor(all_input_vecs[idx])
            if gpu: vecs = vecs.to('cuda')
            y_pred = model(vecs).cpu().detach().numpy()
            pred_labels = [np.argmax(entry) for entry in y_pred]
            predictions += pred_labels
        pre, rec, f1, _ = precision_recall_fscore_support(np.array(labels)[dev_idx], predictions,average='macro')
        print('\n---> after {} epochs, the macro-F1 on dev set is {}'.format(epoch_i,f1))
        print('pred 1 percent', np.sum(predictions)/len(predictions))
        for param_group in optimizer.param_groups:
            print('learning rate', param_group['lr'])
        
        # save the best model
        if f1 > best_f1:
            best_f1 = f1
            best_model = copy.deepcopy(model.state_dict())
            print('best model updated; new best macro-F1',f1)
    
    # (optional) adjust learning rate according to the scheduler
    scheduler.step()


  1%|          | 1/100 [00:00<00:43,  2.27it/s]


======epoch 0 loss====== 0.30748448

---> after 0 epochs, the macro-F1 on dev set is 0.6771713994661194
pred 1 percent 0.13757329724853407
learning rate 6.400000000000001e-05
best model updated; new best macro-F1 0.6771713994661194



  2%|▏         | 2/100 [00:00<00:42,  2.29it/s]


======epoch 1 loss====== 0.30280167

---> after 1 epochs, the macro-F1 on dev set is 0.676318329167801
pred 1 percent 0.13847541723049164
learning rate 6.400000000000001e-05



  3%|▎         | 3/100 [00:01<00:42,  2.26it/s]

...


======epoch 97 loss====== 0.11798878

---> after 97 epochs, the macro-F1 on dev set is 0.7184731086273523
pred 1 percent 0.2169598556608029
learning rate 6.871947673600004e-06



 99%|█████████▉| 99/100 [00:58<00:00,  1.33it/s]


======epoch 98 loss====== 0.117530756

---> after 98 epochs, the macro-F1 on dev set is 0.7184731086273523
pred 1 percent 0.2169598556608029
learning rate 6.871947673600004e-06



100%|██████████| 100/100 [00:59<00:00,  1.68it/s]


======epoch 99 loss====== 0.11708305

---> after 99 epochs, the macro-F1 on dev set is 0.7184731086273523
pred 1 percent 0.2169598556608029
learning rate 6.871947673600004e-06


In [15]:
import numpy as np

model.load_state_dict(best_model)
with torch.no_grad(): # let pytorch know that no gradient should be computed
    model.eval() # let the model know that it in test mode, i.e. no gradient and no dropout
    predictions = []
    for i in range(0,len(test_idx),batch_size):
        idx = test_idx[i:i+batch_size] 
        vecs = torch.tensor(all_input_vecs[idx])
        if gpu: vecs = vecs.to('cuda')
        y_pred = model(vecs).cpu().detach().numpy()
        pred_labels = [np.argmax(entry) for entry in y_pred]
        predictions += pred_labels
    pre, rec, f1, _ = precision_recall_fscore_support(np.array(labels)[test_idx], predictions,average='macro')
    print('\n--->  the macro-F1 on test set is {}'.format(f1))
    print('pred 1 percent', np.sum(predictions)/len(predictions))


--->  the macro-F1 on test set is 0.7126365010256823
pred 1 percent 0.2452667814113597


In [ ]:
# if you want to save your trained model, you may uncomment the line below
# torch.save(best_model, 'bert_pgd_base_wTitle.state_dict')